In [1]:
# Check cuda visible devices
import torch

print(torch.cuda.is_available())

print(torch.cuda.device_count())

True
2


In [3]:
%load_ext autoreload
%autoreload 2
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# custom device map
device_map = {
    'model.norm': 'cpu',
    'lm_head': 'cpu',
    'model.embed_tokens': 0,
}

first_gpu = 14
second_gpu = 28

for i in range(0, first_gpu):
    device_map[f"model.layers.{i}"] = 0

for i in range(first_gpu, second_gpu):
    device_map[f"model.layers.{i}"] = 1

for i in range(second_gpu, 36):
    device_map[f"model.layers.{i}"] = "cpu"

quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    #bnb_4bit_compute_dtype=torch.float16,
    #llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,
    #device_map=device_map,
    #llm_int8_enable_fp32_cpu_offload=True,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: 
                    Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the
                    quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules
                    in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to
                    `from_pretrained`. Check
                    https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                    for more details.
                    

In [ ]:
model.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 1,
 'model.layers.4': 1,
 'model.layers.5': 1,
 'model.layers.6': 1,
 'model.layers.7': 'cpu',
 'model.layers.8': 'cpu',
 'model.layers.9': 'cpu',
 'model.layers.10': 'cpu',
 'model.layers.11': 'cpu',
 'model.layers.12': 'cpu',
 'model.layers.13': 'cpu',
 'model.layers.14': 'cpu',
 'model.layers.15': 'cpu',
 'model.layers.16': 'cpu',
 'model.layers.17': 'cpu',
 'model.layers.18': 'cpu',
 'model.layers.19': 'cpu',
 'model.layers.20': 'cpu',
 'model.layers.21': 'cpu',
 'model.layers.22': 'cpu',
 'model.layers.23': 'cpu',
 'model.layers.24': 'cpu',
 'model.layers.25': 'cpu',
 'model.layers.26': 'cpu',
 'model.layers.27': 'cpu',
 'model.layers.28': 'cpu',
 'model.layers.29': 'cpu',
 'model.layers.30': 'cpu',
 'model.layers.31': 'cpu',
 'model.norm': 'cpu',
 'lm_head': 'cpu'}

In [ ]:
from IPython.display import display, clear_output, HTML
import textwrap

prompt = """<s>[INST] Traducir del español al español rioplatense. Sólo responder la traducción, sin agregar nada más.

Hola hombre, ¿cómo estás?
[/INST]
¿Qué onda, chabón? ¿Cómo va?
</s>
[INST]
Vaya tío, eres un cabrón
[/INST]
Che pedazo de pelotudo, sos un forro
</s>
[INST]
Tu puedes hacerlo!
[/INST]
"""
inputs = tokenizer(prompt, return_tensors="pt")
next_token = None
new_tokens = []
while next_token != tokenizer.eos_token_id:
    output = model.generate(**inputs, max_new_tokens=1, pad_token_id=tokenizer.eos_token_id)

    next_token = output[0][-1].item()
    inputs = {
        "input_ids": output,
        "attention_mask": torch.ones_like(output),
    }
    new_tokens.append(next_token)
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)

    #text = "\n".join(textwrap.wrap(text, width=100))
    clear_output(wait=True)
    display(HTML(text))


/home/jmperez/miniconda3/envs/rioplatense_hs/lib/python3.10/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 66.00 MiB. GPU 1 has a total capacty of 23.53 GiB of which 47.38 MiB is free. Including non-PyTorch memory, this process has 23.48 GiB memory in use. Of the allocated memory 22.24 GiB is allocated by PyTorch, and 236.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
from datasets import load_dataset

dataset_name = "piuba-bigdata/contextualized_hate_speech"

ds = load_dataset(dataset_name)

In [ ]:
test_ds = ds["test"].shuffle()

In [ ]:
from pysentimiento.preprocessing import preprocess_tweet
import re

url_regex = r"\burl\b"


def text_preprocess(text):
    text = preprocess_tweet(text, preprocess_hashtags=False, demoji=False)
    text = text.replace("@usuario", "")

    text = re.sub(url_regex, "", text)
    # Replace multiple spaces with one
    text = " ".join(text.split())
    text = text.replace("\n", " ")

    return text


In [ ]:
tokenizer.model_max_length = 6400
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
from rioplatense_hs.mixtral import get_prompt

def tokenize(example):
    texto = text_preprocess(example["text"])

    contexto = preprocess_tweet(
        example["context_tweet"],
        preprocess_hashtags=False,
        demoji=False,
        preprocess_handles=False,
    )
    model_input = get_prompt(context=contexto, text=texto)
    return tokenizer(model_input, truncation=True)

tokenized_ds = test_ds.map(tokenize, batched=False)

In [ ]:
# prompt: Sort tokenized_ds by length of input_ids
tokenized_ds = tokenized_ds.map(lambda x: {"len": len(x["input_ids"])}, batched=False)

sorted_tokenized_ds = tokenized_ds.sort("len")

In [ ]:
# decode first example

inputs = sorted_tokenized_ds[0]["input_ids"]

text = tokenizer.decode(inputs, skip_special_tokens=True)

text = text.replace("[INST]", "\n[INST]")
text = text.replace("[/INST]", "[/INST]\n")

print(text)

In [ ]:
# prompt: Build a dataloader with a Causal LLM collator

from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
batch_size = 16
collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

def collate(inputs):
    attention = [ex["attention_mask"] for ex in inputs]
    input_ids = [ex["input_ids"] for ex in inputs]
    ids = [ex["id"] for ex in inputs]
    return ids, tokenizer.pad({"input_ids": input_ids, "attention_mask": attention}, return_tensors="pt")


dataloader = DataLoader(
    sorted_tokenized_ds,
    batch_size=batch_size,
    collate_fn=collate,
    shuffle=False,
    pin_memory=True,
    num_workers=16,
)


In [ ]:
from tqdm.auto import tqdm


for ids, inputs in tqdm(dataloader):
    output = model.generate(**inputs, max_new_tokens=150)

    for k, id in enumerate(ids):
        output_text = tokenizer.decode(output[k], skip_special_tokens=True)

        #with open(path+"/{}.txt".format(id), "w") as f:
        #    f.write(output_text)

    #for _ in range(4):
    #    print("="*80)
    #print(f"contexto: {contexto}")
    #print(f"texto: {texto}")
    #print(output_text)
